In [15]:
import numpy as np
import pandas as pd
import re
import math

In [16]:
df = pd.read_csv('AmazonPay3.csv')

#Normalize data, drop tweets that contain order ids, reference numbers, and complaint numbers

In [17]:
df.Text = df['Text'].str.lower()
df_temp = df.drop(df[df.Text.str.contains('order')].index)
df_temp = df_temp.drop(df_temp[df_temp.Text.str.contains('ref')].index)
df_filtered = df_temp.drop(df_temp[df_temp.Text.str.contains('complaint no')].index)

In [18]:
df_filtered.shape

(168, 7)

In [19]:
col_list =["Text"]
numbers = []
for index,row in df_filtered['Text'].iteritems():
    numbers.append(re.findall(r'\d{10}',str(row)))

#make a dictionary for scanned numbers so that numbers that appear more than once have an increased score

In [20]:
numbers_scanned = {}

In [21]:
for s in numbers:
    for j in s:
        if int(j)>6000000000:
            if j in numbers_scanned:
                numbers_scanned[j]+=1
            else:
                numbers_scanned[j] = 1

In [22]:
print(numbers_scanned)

{'9883825160': 54, '9414431349': 1, '9126362188': 1, '8201919360': 1, '6014854711': 1, '9042669091': 1, '8319895945': 3, '9323029393': 1, '8201913795': 1, '8201917631': 1, '8201917620': 1, '8201916777': 1, '8201915315': 1, '6357000145': 2, '8201914435': 1, '8602062739': 1, '8201913662': 1, '9410832123': 5, '8201910780': 1, '8800561110': 1, '9910920899': 1, '8201912420': 2, '8201908853': 1, '8201912562': 1, '9063388404': 1, '8201910893': 1, '8201910106': 1, '7038837237': 1, '9813833298': 1, '8201909716': 1, '8201910604': 8, '9500515507': 1, '9641396199': 1, '8201909313': 1, '8514948806': 1, '9404685034': 1, '8690016166': 1, '9999781506': 1, '9419181965': 1, '8294842724': 81, '8597857956': 37, '9080782927': 1, '9039112181': 1, '7524802317': 1, '8977186144': 1, '9931119848': 3, '8201904587': 1, '9967646355': 1, '8201902366': 1, '8201902483': 1, '9883322075': 18, '8389862296': 15, '7865872004': 15, '7860788671': 1, '8101919119': 1, '9818206738': 1, '8201901984': 1, '7988974016': 1, '620227

In [23]:
res = 0
for val in numbers_scanned.values():
    res += val

res = res / len(numbers_scanned)

#K is the average of how many times a number appears, based on that classify numbers as flagged or not

In [24]:
res = math.floor(res)

In [25]:
flagged_numbers = [k for k in numbers_scanned if numbers_scanned[k] >= res]

In [26]:
print(flagged_numbers)

['9883825160', '8319895945', '9410832123', '8201910604', '8294842724', '8597857956', '9931119848', '9883322075', '8389862296', '7865872004', '6202278073', '9875331783', '9330340020', '9113761215']


In [31]:
print(len(flagged_numbers))

14


#Since Amazon pay has no numbers affiliated with them, this array is empty

In [27]:
AmazonPay_numbers = []

In [28]:
for number in AmazonPay_numbers:
    if number in flagged_numbers:
        flagged_numbers.remove(number)


#Sort the numbers by the number of followers they got posted by (lowest follower = higher chance of being fake)

In [29]:
found = False
for row in df.itertuples():
    found = False
    Tweet = str(row.Text)
    for num in flagged_numbers:
        if num in Tweet:
            found = True
    if not found:
        df = df.drop([row.Index])

In [30]:
df.to_csv('FraudAmazonPay.csv', sep=',')